In [ ]:
# 1. Install libs and set python env

In [ ]:
!pip install --upgrade pip
!pip install nncase --timeout=1000
!pip install nncase-kpu --timeout=1000
!pip install onnx onnxsim scikit-learn

# nncase-2.x need dotnet-7
# Ubuntu use apt to install dotnet-7.0 (The docker has installed dotnet7.0)
!sudo apt-get install -y dotnet-sdk-7.0

In [ ]:
%env PATH=$PATH:/usr/local/lib/python3.8/site-packages

In [ ]:
# 2. Set compile options and PTQ options (quantize model)

In [ ]:
import nncase
import numpy as np
from nncase_base_func import *

def compile_kmodel(model_path, dump_path, calib_data):
    """
    Set compile options and ptq options.
    Compile kmodel.
    Dump the compile-time result to 'compile_options.dump_dir'
    """
    print("\n----------   compile    ----------")
    print("Simplify...")
    model_file = model_simplify(model_path)

    print("Set options...")
    # import_options
    import_options = nncase.ImportOptions()

    ############################################
    # The code below, you need to modify to fit your model
    ############################################
    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = "k230" #"cpu"
    compile_options.dump_ir = True  # if False, will not dump the compile-time result.
    compile_options.dump_asm = True
    compile_options.dump_dir = dump_path
    compile_options.input_file = ""
    
    # preprocess args
    # more details in docs/USAGE_v2.md
    compile_options.preprocess = False
    if compile_options.preprocess:
        compile_options.input_type = "uint8" # "uint8" "float32"
        compile_options.input_shape = [1,224,320,3]
        compile_options.input_range = [0,1]
        compile_options.input_layout = "NHWC" # "NHWC"
        compile_options.swapRB = False
        compile_options.mean = [0,0,0]
        compile_options.std = [1,1,1]
        compile_options.letterbox_value = 0
        compile_options.output_layout = "NHWC" # "NHWC"
    # quant options
    ptq_options = nncase.PTQTensorOptions()
    ptq_options.quant_type = "uint8" # datatype : "float32", "int8", "int16"
    ptq_options.w_quant_type = "uint8"  # datatype : "float32", "int8", "int16"
    ptq_options.calibrate_method = "NoClip" # "Kld"
    ptq_options.finetune_weights_method = "NoFineTuneWeights"
    ptq_options.dump_quant_error = False
    ptq_options.dump_quant_error_symmetric_for_signed = False
    
    # more details in docs/MixQuant.md
    ptq_options.quant_scheme = ""
    ptq_options.export_quant_scheme = False
    ptq_options.export_weight_range_by_channel = False
    ############################################
    
    ptq_options.samples_count = len(calib_data[0])
    ptq_options.set_tensor_data(calib_data)
    
    print("Compiling...")
    compiler = nncase.Compiler(compile_options)
    # import
    model_content = read_model_file(model_file)
    if model_path.split(".")[-1] == "onnx":
        compiler.import_onnx(model_content, import_options)
    elif model_path.split(".")[-1] == "tflite":
        compiler.import_tflite(model_content, import_options)
    
    compiler.use_ptq(ptq_options)
    
    # compile
    compiler.compile()
    kmodel = compiler.gencode_tobytes()
    
    kmodel_path = os.path.join(dump_path, "test.kmodel")
    with open(kmodel_path, 'wb') as f:
        f.write(kmodel)
    print("----------------end-----------------")
    return kmodel_path


In [ ]:
# 3. Example of compiling model with a single input.

In [ ]:
# compile kmodel single input
model_path = "./test.tflite"
dump_path = "./tmp_tflite"

# If model has multiple inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'

# sample_count is 2
calib_data = [[np.random.rand(1, 240, 320, 3).astype(np.float32), np.random.rand(1, 240, 320, 3).astype(np.float32)]]

kmodel_path = compile_kmodel(model_path, dump_path, calib_data)


In [ ]:
# 4. Example of simulating kmodel with a single input.

In [ ]:
# run kmodel(simulate)
import os

kmodel_path = "./tmp_tflite/test.kmodel"
input_data = [np.random.rand(1, 240, 320, 3).astype(np.float32)]

result = run_kmodel(kmodel_path, input_data)

for idx, i in enumerate(result):
    print(i.shape)
    i.tofile(os.path.join(dump_path,"nncase_result_{}.bin".format(idx)))

In [ ]:
# 5. Example of comparing kmodel result with tflite result.

In [ ]:
!pip install tensorflow
import tensorflow as tf
interp = tf.lite.Interpreter(model_path=model_path)

inputs = []
for idx, item in enumerate(interp.get_input_details()):
    input_dict = {}
    input_dict['index'] = item['index']
    input_dict['value'] = input_data[idx]
    inputs.append(input_dict)
    
# print(input_dict)

interp.allocate_tensors()
for input in inputs:
    interp.set_tensor(input['index'], input['value'])
interp.invoke()

tflite_result = []
for item in interp.get_output_details():
    tflite_result.append(interp.get_tensor(item['index']))

for index, (i, j) in enumerate(zip(tflite_result, result)):
    print("result {} cosine = ".format(index), get_cosine(i, j))


In [ ]:
# 6. Example of compiling model with multiple inputs.

In [ ]:
# compile kmodel multiple inputs
model_path = "./test.onnx"
dump_path = "./tmp_onnx"

# If model has multiple inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'
# x:{shape: [3,100], range: [1,5], dtype: int64}
# y:{shape: [100, 3, 192], range: [0,1), dtype: float32}
# z:{shape: [3,100], dtype: bool}
# calib_data = [[ np.random.randint(1, 5, size=[3,100], dtype='int64'), np.random.randint(1, 5, size=[3,100], dtype='int64')],
#               [ np.random.rand(100, 3, 192).astype(np.float32), np.random.rand(100, 3, 192).astype(np.float32)],
#               [ np.random.rand(3,100).astype(np.float32)>0.5, np.random.rand(3,100).astype(np.float32)>0.5],] # bool

# sample_count is 2
calib_data = [[np.random.rand(1, 1, 1024).astype(np.float32), np.random.rand(1, 1, 1024).astype(np.float32)],
              [np.random.rand(1, 1, 320).astype(np.float32), np.random.rand(1, 1, 320).astype(np.float32)]]

kmodel_path = compile_kmodel(model_path, dump_path, calib_data)



In [ ]:
# 7. Example of simulating kmodel with multiple inputs.

In [ ]:
# run kmodel(simulate)
import os

kmodel_path = "./tmp_onnx/test.kmodel"
input_data = [np.random.rand(1, 1, 1024).astype(np.float32), np.random.rand(1, 1, 320).astype(np.float32)]

results = run_kmodel(kmodel_path, input_data)

for idx, i in enumerate(results):
    print(i.shape)
    i.tofile(os.path.join(dump_path,"nncase_result_{}.bin".format(idx)))


In [ ]:
# 8. Example of comparing kmodel results with onnx results.

In [ ]:
!pip install onnxruntime
import onnxruntime as rt

# onnx_model = model_simplify(model_path)
onnx_model = model_path
_, input_info = parse_model_input_output(model_path)
onnx_sess = rt.InferenceSession(onnx_model)

input_dict = {}
for i, info in enumerate(input_info):
    print(info['shape'])
    input_dict[info["name"]] = input_data[i]

onnx_results = onnx_sess.run(None, input_dict)
for index, (i, j) in enumerate(zip(onnx_results, results)):
    print("result {} cosine = ".format(index), get_cosine(i, j))